# ETL: Transformación y Unificación de Datasets de Felicidad (2015–2019)

Este notebook implementa el proceso de limpieza, estandarización y unión de los datasets del World Happiness Report para obtener un único conjunto de datos listo para modelado.

In [1]:
# Importar librerías
import pandas as pd
import numpy as np

## 1. Cargar los datasets originales
Cargamos los archivos CSV de los años 2015 a 2019.

In [2]:
# Cargar los datasets
files = {
    2015: '../data/2015.csv',
    2016: '../data/2016.csv',
    2017: '../data/2017.csv',
    2018: '../data/2018.csv',
    2019: '../data/2019.csv'
}
dfs = {year: pd.read_csv(path) for year, path in files.items()}

In [3]:
# Analizar nombres de columnas en cada dataset
for year in dfs:
    print(f"\nColumnas en dataset {year}:")
    print(dfs[year].columns.tolist())


Columnas en dataset 2015:
['Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Standard Error', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual']

Columnas en dataset 2016:
['Country', 'Region', 'Happiness Rank', 'Happiness Score', 'Lower Confidence Interval', 'Upper Confidence Interval', 'Economy (GDP per Capita)', 'Family', 'Health (Life Expectancy)', 'Freedom', 'Trust (Government Corruption)', 'Generosity', 'Dystopia Residual']

Columnas en dataset 2017:
['Country', 'Happiness.Rank', 'Happiness.Score', 'Whisker.high', 'Whisker.low', 'Economy..GDP.per.Capita.', 'Family', 'Health..Life.Expectancy.', 'Freedom', 'Generosity', 'Trust..Government.Corruption.', 'Dystopia.Residual']

Columnas en dataset 2018:
['Overall rank', 'Country or region', 'Score', 'GDP per capita', 'Social support', 'Healthy life expectancy', 'Freedom to make life choices', 'Generosity', 'Perceptions of corruption'

## 2. Diccionarios de estandarización
Definimos los diccionarios para renombrar columnas y normalizar nombres de países.

In [4]:
# Diccionario para renombrar columnas equivalentes
column_map = {
    # GDP per Capita
    'Economy (GDP per Capita)': 'GDP_per_Capita',
    'Economy..GDP.per.Capita.': 'GDP_per_Capita',
    'GDP per capita': 'GDP_per_Capita',
    
    # Social Support
    'Family': 'Social_Support',
    'Social support': 'Social_Support',
    
    # Healthy Life Expectancy
    'Health (Life Expectancy)': 'Healthy_Life_Expectancy',
    'Health..Life.Expectancy.': 'Healthy_Life_Expectancy',
    'Healthy life expectancy': 'Healthy_Life_Expectancy',
    
    # Freedom
    'Freedom': 'Freedom',
    'Freedom to make life choices': 'Freedom',
    
    # Generosity
    'Generosity': 'Generosity',
    
    # Perceptions of Corruption
    'Trust (Government Corruption)': 'Perceptions_of_Corruption',
    'Perceptions of corruption': 'Perceptions_of_Corruption',
    'Trust..Government.Corruption.': 'Perceptions_of_Corruption',
    
    # Happiness Score
    'Happiness Score': 'Happiness_Score',
    'Score': 'Happiness_Score',
    'Happiness.Score': 'Happiness_Score',

    # Country
    'Country': 'Country',
    'Country or region': 'Country',
    
    # Region
    'Region': 'Region',
}

# Diccionario de alias para normalizar nombres de países
country_alias = {
    # Variaciones de escritura y símbolos
    "Trinidad & Tobago": "Trinidad and Tobago",
    "Taiwan Province of China": "Taiwan",
    "Hong Kong S.A.R., China": "Hong Kong",
    "Northern Cyprus": "North Cyprus",
    "North Macedonia": "Macedonia",
    "Congo (Brazzaville)": "Congo",
    "Congo (Kinshasa)": "Democratic Republic of the Congo",
    "Ivory Coast": "Côte d’Ivoire",
    "Swaziland": "Eswatini",
    "Eswatini": "Eswatini",
    "Gambia": "Gambia",
    "Somaliland Region": "Somaliland region",
    "Somaliland region": "Somaliland region",
}


## 3. Creación del diccionario maestro de regiones
Creamos un diccionario de regiones usando los datos de 2015 y 2016 que contienen la información regional completa.

In [5]:
# Crear diccionario de regiones a partir de 2015 y 2016
region_map = {}

# Extraer información de regiones de 2015 y 2016
for year in [2015, 2016]:
    df = dfs[year]
    for _, row in df.iterrows():
        country = row['Country']
        # Normalizar el nombre del país si existe un alias
        if country in country_alias:
            country = country_alias[country]
        # Guardar la región correspondiente
        if 'Region' in df.columns and not pd.isna(row['Region']):
            region_map[country] = row['Region']

print("Número de países con región asignada:", len(region_map))
print("\nEjemplo de algunas asignaciones de región:")
for country in list(region_map.keys())[:5]:
    print(f"{country}: {region_map[country]}")

Número de países con región asignada: 163

Ejemplo de algunas asignaciones de región:
Switzerland: Western Europe
Iceland: Western Europe
Denmark: Western Europe
Norway: Western Europe
Canada: North America


## 4. Estandarización y limpieza de datasets
Procesamos cada dataset para:
1. Renombrar columnas según el diccionario de estandarización
2. Normalizar nombres de países
3. Asignar regiones faltantes
4. Seleccionar solo las columnas necesarias
5. Agregar columna de año

In [6]:
# Columnas que queremos mantener en el dataset final
columns_to_keep = [
    'Country', 'Region', 'Year', 'Happiness_Score',
    'GDP_per_Capita', 'Social_Support', 'Healthy_Life_Expectancy',
    'Freedom', 'Generosity', 'Perceptions_of_Corruption'
]

# Procesar cada dataset
cleaned_dfs = {}
for year, df in dfs.items():
    # 1. Renombrar columnas
    df = df.rename(columns=column_map)
    
    # 2. Normalizar nombres de países
    df['Country'] = df['Country'].replace(country_alias)
    
    # 3. Asignar regiones faltantes
    if 'Region' not in df.columns:
        df['Region'] = df['Country'].map(region_map)
    
    # 4. Agregar columna de año
    df['Year'] = year
    
    # 5. Seleccionar solo las columnas necesarias
    # Primero, asegurémonos de que todas las columnas necesarias existen
    for col in columns_to_keep:
        if col not in df.columns:
            print(f"Advertencia: Columna {col} no encontrada en el dataset de {year}")
    
    # Seleccionar solo las columnas que existen
    available_columns = [col for col in columns_to_keep if col in df.columns]
    cleaned_df = df[available_columns]
    
    # Guardar el dataset limpio
    cleaned_dfs[year] = cleaned_df
    
    # Imprimir información sobre el proceso
    print(f"\nDataset {year}:")
    print(f"Dimensiones: {cleaned_df.shape}")
    print(f"Columnas: {cleaned_df.columns.tolist()}")
    print(f"Valores nulos: {cleaned_df.isnull().sum().sum()}")


Dataset 2015:
Dimensiones: (158, 10)
Columnas: ['Country', 'Region', 'Year', 'Happiness_Score', 'GDP_per_Capita', 'Social_Support', 'Healthy_Life_Expectancy', 'Freedom', 'Generosity', 'Perceptions_of_Corruption']
Valores nulos: 0

Dataset 2016:
Dimensiones: (157, 10)
Columnas: ['Country', 'Region', 'Year', 'Happiness_Score', 'GDP_per_Capita', 'Social_Support', 'Healthy_Life_Expectancy', 'Freedom', 'Generosity', 'Perceptions_of_Corruption']
Valores nulos: 0

Dataset 2017:
Dimensiones: (155, 10)
Columnas: ['Country', 'Region', 'Year', 'Happiness_Score', 'GDP_per_Capita', 'Social_Support', 'Healthy_Life_Expectancy', 'Freedom', 'Generosity', 'Perceptions_of_Corruption']
Valores nulos: 0

Dataset 2018:
Dimensiones: (156, 10)
Columnas: ['Country', 'Region', 'Year', 'Happiness_Score', 'GDP_per_Capita', 'Social_Support', 'Healthy_Life_Expectancy', 'Freedom', 'Generosity', 'Perceptions_of_Corruption']
Valores nulos: 1

Dataset 2019:
Dimensiones: (156, 10)
Columnas: ['Country', 'Region', 'Year'

## 4.1 Redondeo de valores numéricos
Estandarizamos todos los valores numéricos a 4 decimales para mantener consistencia entre todos los años.

In [7]:
# Columnas numéricas que necesitan redondeo
numeric_columns = [
    'Happiness_Score',
    'GDP_per_Capita',
    'Social_Support',
    'Healthy_Life_Expectancy',
    'Freedom',
    'Generosity',
    'Perceptions_of_Corruption'
]

# Aplicar redondeo a 3 decimales en cada dataset limpio
for year in cleaned_dfs:
    for col in numeric_columns:
        if col in cleaned_dfs[year].columns:
            cleaned_dfs[year][col] = cleaned_dfs[year][col].round(3)
            
# Regenerar el dataset concatenado con los valores redondeados
happiness_all = pd.concat(cleaned_dfs.values(), ignore_index=True)

# Verificar los cambios
print("Muestra de valores redondeados para cada columna numérica:")
for col in numeric_columns:
    if col in happiness_all.columns:
        print(f"\n{col}:")
        print(happiness_all[col].head())

Muestra de valores redondeados para cada columna numérica:

Happiness_Score:
0    7.587
1    7.561
2    7.527
3    7.522
4    7.427
Name: Happiness_Score, dtype: float64

GDP_per_Capita:
0    1.397
1    1.302
2    1.325
3    1.459
4    1.326
Name: GDP_per_Capita, dtype: float64

Social_Support:
0    1.350
1    1.402
2    1.361
3    1.331
4    1.323
Name: Social_Support, dtype: float64

Healthy_Life_Expectancy:
0    0.941
1    0.948
2    0.875
3    0.885
4    0.906
Name: Healthy_Life_Expectancy, dtype: float64

Freedom:
0    0.666
1    0.629
2    0.649
3    0.670
4    0.633
Name: Freedom, dtype: float64

Generosity:
0    0.297
1    0.436
2    0.341
3    0.347
4    0.458
Name: Generosity, dtype: float64

Perceptions_of_Corruption:
0    0.420
1    0.141
2    0.484
3    0.365
4    0.330
Name: Perceptions_of_Corruption, dtype: float64


C:\Users\fabia\AppData\Local\Temp\ipykernel_10640\299870449.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_dfs[year][col] = cleaned_dfs[year][col].round(3)


In [8]:
# Concatenar todos los datasets limpios
happiness_all = pd.concat(cleaned_dfs.values(), ignore_index=True)

# Verificar el resultado final
print("Dimensiones del dataset final:", happiness_all.shape)
print("\nColumnas del dataset final:", happiness_all.columns.tolist())
print("\nDistribución por año:")
print(happiness_all['Year'].value_counts().sort_index())
print("\nValores nulos por columna:")
print(happiness_all.isnull().sum())

Dimensiones del dataset final: (782, 10)

Columnas del dataset final: ['Country', 'Region', 'Year', 'Happiness_Score', 'GDP_per_Capita', 'Social_Support', 'Healthy_Life_Expectancy', 'Freedom', 'Generosity', 'Perceptions_of_Corruption']

Distribución por año:
Year
2015    158
2016    157
2017    155
2018    156
2019    156
Name: count, dtype: int64

Valores nulos por columna:
Country                      0
Region                       1
Year                         0
Happiness_Score              0
GDP_per_Capita               0
Social_Support               0
Healthy_Life_Expectancy      0
Freedom                      0
Generosity                   0
Perceptions_of_Corruption    1
dtype: int64


## 5. Concatenación y exportación final
Unimos todos los datasets limpios en uno solo y lo exportamos.

In [9]:
# Exportar datasets limpios individuales
for year, df in cleaned_dfs.items():
    output_path = f'../data/clean/clean_{year}.csv'
    df.to_csv(output_path, index=False)
    print(f"Dataset limpio de {year} guardado en: {output_path}")

# Exportar dataset final concatenado
final_output_path = '../data/happiness_all.csv'
happiness_all.to_csv(final_output_path, index=False)
print(f"\nDataset final guardado en: {final_output_path}")

Dataset limpio de 2015 guardado en: ../data/clean/clean_2015.csv
Dataset limpio de 2016 guardado en: ../data/clean/clean_2016.csv
Dataset limpio de 2017 guardado en: ../data/clean/clean_2017.csv


Dataset limpio de 2018 guardado en: ../data/clean/clean_2018.csv
Dataset limpio de 2019 guardado en: ../data/clean/clean_2019.csv

Dataset final guardado en: ../data/happiness_all.csv


## 6. Corrección de valores faltantes
Manejamos dos casos especiales:
1. Asignación de región faltante para Gambia
2. Imputación del valor faltante en Perceptions_of_Corruption

In [10]:
df = pd.read_csv("../data/happiness_all.csv")

# Completar región faltante para Gambia (añadida en 2019)
df.loc[df["Country"] == "Gambia", "Region"] = "Sub-Saharan Africa"

# Identificar la fila con el valor faltante
missing_row = df[df["Perceptions_of_Corruption"].isna()]
print("Fila con valor faltante:\n", missing_row[["Country", "Region", "Year"]])

# Extraer región y año del registro faltante
region_missing = missing_row["Region"].values[0]
year_missing = missing_row["Year"].values[0]

# Calcular la media de la región y año correspondiente
mean_value = df[
    (df["Region"] == region_missing) & (df["Year"] == year_missing)
]["Perceptions_of_Corruption"].mean()

# Redondear la media a tres decimales antes de imputar
mean_value = round(mean_value, 3)
print(f"\nMedia de corrupción en {region_missing} ({year_missing}): {mean_value}")

# Imputar el valor faltante con la media calculada
df.loc[df["Perceptions_of_Corruption"].isna(), "Perceptions_of_Corruption"] = mean_value

# Asegurar que toda la columna tenga tres decimales
df["Perceptions_of_Corruption"] = df["Perceptions_of_Corruption"].round(3)

df.to_csv("../data/happiness_all.csv", index=False, encoding="utf-8")
print("Dataset final validado y completo.")


Fila con valor faltante:
                   Country                           Region  Year
489  United Arab Emirates  Middle East and Northern Africa  2018

Media de corrupción en Middle East and Northern Africa (2018): 0.123
Dataset final validado y completo.


In [11]:
print("Dimensiones:", df.shape)
print("\nColumnas:", df.columns.tolist())
print("\nTipos de datos:")
print(df.dtypes)

Dimensiones: (782, 10)

Columnas: ['Country', 'Region', 'Year', 'Happiness_Score', 'GDP_per_Capita', 'Social_Support', 'Healthy_Life_Expectancy', 'Freedom', 'Generosity', 'Perceptions_of_Corruption']

Tipos de datos:
Country                       object
Region                        object
Year                           int64
Happiness_Score              float64
GDP_per_Capita               float64
Social_Support               float64
Healthy_Life_Expectancy      float64
Freedom                      float64
Generosity                   float64
Perceptions_of_Corruption    float64
dtype: object


In [12]:
print("Valores nulos por columna:")
print(df.isna().sum())

dupes = df.duplicated(subset=["Country", "Year"]).sum()
print(f"\nDuplicados por (Country, Year): {dupes}")


Valores nulos por columna:
Country                      0
Region                       0
Year                         0
Happiness_Score              0
GDP_per_Capita               0
Social_Support               0
Healthy_Life_Expectancy      0
Freedom                      0
Generosity                   0
Perceptions_of_Corruption    0
dtype: int64

Duplicados por (Country, Year): 0


In [13]:
cols = ["Happiness_Score","GDP_per_Capita","Social_Support",
        "Healthy_Life_Expectancy","Freedom","Generosity",
        "Perceptions_of_Corruption"]

df[cols].describe().T


,count,mean,std,min,25%,50%,75%,max
Happiness_Score,782.0,5.379018,1.127456,2.693,4.50975,5.3220,6.18950,7.769
GDP_per_Capita,782.0,0.916040,0.407357,0.000,0.60650,0.9820,1.23625,2.096
Social_Support,782.0,1.078395,0.329538,0.000,0.86925,1.1245,1.32725,1.644
Healthy_Life_Expectancy,782.0,0.612417,0.248298,0.000,0.44025,0.6470,0.80800,1.141
Freedom,782.0,0.411090,0.152891,0.000,0.31000,0.4310,0.53100,0.724
Generosity,782.0,0.218579,0.122336,0.000,0.13000,0.2020,0.27875,0.838
Perceptions_of_Corruption,782.0,0.125423,0.105742,0.000,0.05425,0.0910,0.15575,0.552


In [14]:
print("Años únicos:", sorted(df["Year"].unique()))
print("\nRegiones únicas:", df["Region"].nunique())
print(df["Region"].value_counts())


Años únicos: [2015, 2016, 2017, 2018, 2019]

Regiones únicas: 10
Region
Sub-Saharan Africa                 196
Central and Eastern Europe         145
Latin America and Caribbean        111
Western Europe                     105
Middle East and Northern Africa     96
Southeastern Asia                   44
Southern Asia                       35
Eastern Asia                        30
North America                       10
Australia and New Zealand           10
Name: count, dtype: int64
